In [1]:
import os
import zipfile
import csv

from collections import defaultdict, Counter

In [2]:
path_to_data = os.path.join("data", "input")
path_out = os.path.join("data", "processed.csv")

In [3]:
# to_exclude = ["18_0{}.zip".format(x) for x in range(1, 6)]
to_exclude = []

In [ ]:
%%time
betreiber_names_per_id = defaultdict(Counter)
haltestellen_names_per_uic = defaultdict(Counter)
with open(path_out, "wb") as out:
    csv_writer = csv.writer(out, delimiter=";")
    header = ["BETRIEBSTAG", "FAHRT_BEZEICHNER", "BETREIBER_ID", "VERKEHRSMITTEL_TEXT", "BPUIC", "ANKUNFTSZEIT", "AN_PROGNOSE", "ABFAHRTSZEIT", "AB_PROGNOSE"]
    csv_writer.writerow(header)
    for filename in os.listdir(path_to_data):
        if filename not in to_exclude:
            if filename.endswith(".zip"):
                print filename
                with zipfile.ZipFile(os.path.join(path_to_data, filename)) as f:
                    for name in f.namelist():
                        with f.open(name) as ff:
                            reader = csv.DictReader(ff, delimiter=";")
                            nb_written = 0
                            for line in reader:
                                produkt = line['PRODUKT_ID']
                                an_status = line['AN_PROGNOSE_STATUS']
                                ab_status = line['AB_PROGNOSE_STATUS']
                                if produkt == 'Zug' and ((an_status == 'REAL') or (ab_status == 'REAL')):
                                    bpuic = line["BPUIC"]
                                    haltestellen_names_per_uic[bpuic][line["HALTESTELLEN_NAME"]] += 1
                                    betreiber_id = line["BETREIBER_ID"]
                                    betreiber_names_per_id[betreiber_id][line["BETREIBER_NAME"]] += 1
                                    an = line['AN_PROGNOSE']
                                    an_real = an if an_status == "REAL" else ""
                                    ab = line['AB_PROGNOSE']
                                    ab_real = ab if ab_status == "REAL" else ""
                                    csv_writer.writerow([
                                        line["BETRIEBSTAG"], 
                                        line["FAHRT_BEZEICHNER"], 
                                        line["BETREIBER_ID"], 
                                        line["VERKEHRSMITTEL_TEXT"],
                                        bpuic,
                                        line["ANKUNFTSZEIT"],
                                        an_real,
                                        line["ABFAHRTSZEIT"],
                                        ab_real
                                    ])
                                    nb_written += 1
                        out.flush()
                        print "   {} processed. # lines written: {}".format(name, nb_written)

18_01.zip
   jan18/2018-01-01istdaten.csv processed. # lines written: 0
   jan18/2018-01-02istdaten.csv processed. # lines written: 0
   jan18/2018-01-03istdaten.csv processed. # lines written: 0
   jan18/2018-01-04istdaten.csv processed. # lines written: 0
   jan18/2018-01-05istdaten.csv processed. # lines written: 0
   jan18/2018-01-06istdaten.csv processed. # lines written: 0
   jan18/2018-01-07istdaten.csv processed. # lines written: 0
   jan18/2018-01-08istdaten.csv processed. # lines written: 0
   jan18/2018-01-09istdaten.csv processed. # lines written: 0
   jan18/2018-01-10istdaten.csv processed. # lines written: 0
   jan18/2018-01-11istdaten.csv processed. # lines written: 0
   jan18/2018-01-12istdaten.csv processed. # lines written: 0
   jan18/2018-01-13istdaten.csv processed. # lines written: 0
   jan18/2018-01-14istdaten.csv processed. # lines written: 0
   jan18/2018-01-15istdaten.csv processed. # lines written: 0
   jan18/2018-01-16istdaten.csv processed. # lines written: 

In [ ]:
def write_to_csv(counter_per_id, header, path_out):
    with open(path_out, "wb") as f:
        writer = csv.writer(f, delimiter=";")
        writer.writerow(header)
        for _id, counter in counter_per_id.items():
            for i, (k, v) in enumerate(counter.items()):
                writer.writerow([_id, i, k, v])

In [ ]:
%%time
path_out_betreiber = os.path.join("data", "betreiber.csv")
header_betreiber = ["BETREIBER_ID", "INDEX", "BETREIBER_NAME", "COUNT"]
write_to_csv(betreiber_names_per_id, header_betreiber, path_out_betreiber)

In [ ]:
%%time
path_out_haltestellen = os.path.join("data", "haltestellen.csv")
header_haltestellenr = ["BPUIC", "INDEX", "HALTESTELLEN_NAME", "COUNT"]
write_to_csv(haltestellen_names_per_uic, header_haltestellenr, path_out_haltestellen)